In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
%cd /content/drive/MyDrive/AI/match_naming/
!ls

/content/drive/MyDrive/AI/match_naming
catboost_info  data   labeled	  mapping.json	python-poppler
checkpoints    files  main.ipynb  mapping.pkl	train.zip


In [2]:
!pip install pdfminer.six

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.2 MB/s eta 0:00:00


In [4]:
import re
!pip install Levenshtein -qqq
from Levenshtein import distance

def ngrams(text, n):
    words = re.findall(r'\b\w+\b', text)
    return [' '.join(words[i:i+n]) for i in range(len(words)-(n-1))]

def get_similar_strings(target_string, text, n, max_distance):
    # Create n-grams of the text
    text_ngrams = ngrams(text, n)

    similar_strings = []
    for ngram in text_ngrams:
        # Calculate the Levenshtein distance
        lev_distance = distance(target_string, ngram)
        if lev_distance <= max_distance:
            similar_strings.append((ngram, lev_distance))

    return similar_strings

# Testing
target_string = "Project X"
text = "This document pertains to project X. The information in this document is relevant to projects such as Project x, Project Y, and Project Z. However, please note that the data may not be accurate for projectx or project Y."
n = len(target_string.split())
max_distance = 2
similar_strings = get_similar_strings(target_string, text, n, max_distance)
print(similar_strings)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.5/172.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 45.3 MB/s eta 0:00:00
[('project X', 1), ('Project x', 1), ('Project Y', 1), ('Project Z', 1), ('project Y', 2)]


In [ ]:
# 1. Название может быть в 1-2 строчках
# 2. Может быть в << >> | Но в кавычках может быть внутри названия названия
# 3. Всегда жирным
# 4. Titles can be images so we can OCR them

# Мб учитывать длину строки
# Мб достать font size

In [ ]:
# Image instead of titile counter = 2

In [ ]:
# Title in several lines counter = 3 | So we can join them by font size (but need to be accurate)

In [ ]:
# Title may incldue Этап = 2

In [ ]:
# Whole file is a scan = 1

In [ ]:
from pdfminer.high_level import extract_pages
import glob
files = glob.glob('data/project1/*.pdf')
files[0]

for file in files:
    print(file)
    for page_layout in extract_pages(file):
        for element in page_layout:
            print(element)
        break
    print('\n'*5)

In [ ]:
# from pdfminer.pdfparser import PDFParser
# from pdfminer.pdfdocument import PDFDocument
# from pdfminer.pdftypes import PDFObjectNotFound



# def extract(file_name):
#     pages = []
#     result = []
#     fp = open(file_name, 'rb')
#     parser = PDFParser(fp)
#     doc = PDFDocument(parser, "")
#     visited = set()
#     for xref in doc.xrefs:
#         # print('xref ', xref)
#         for objid in xref.get_objids():
#             if objid in visited: continue
#             visited.add(objid)

#             obj = doc.getobj(objid)
#             if obj is None: continue
#             if isinstance(obj, dict):
#                 # 'Type' is PDFObjRef type
#                 if 'Type' in obj and obj['Type'].name == 'Page':
#                     pages.append(objid)
#                 elif 'C' in obj:
#                     result.append(obj['Rect'])
#     return result
# extract(files[0])

# for file in files:
#     print(file)
#     rects = extract(file)
#     for rect in rects:
#         extracted_text = extract_text_by_coordinates(file,rect,)
#         print(extracted_text)

#     print()

# from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
# from pdfminer.converter import PDFPageAggregator
# from pdfminer.layout import LAParams, LTTextBox, LTTextBoxHorizontal
# from pdfminer.pdfpage import PDFPage

# def extract_text_by_coordinates(pdf_path, coordinates):
#     # Coordinates of the rectangle
#     x1, y1, x2, y2 = coordinates

#     # Open the pdf file
#     fp = open(pdf_path, 'rb')

#     # Create a PDF resource manager object that stores shared resources.
#     rsrcmgr = PDFResourceManager()

#     # Create a PDF device object.
#     device = PDFPageAggregator(rsrcmgr, laparams=LAParams())

#     # Create a PDF interpreter object.
#     interpreter = PDFPageInterpreter(rsrcmgr, device)

#     # Process each page contained in the document.
#     for page in PDFPage.get_pages(fp):
#         interpreter.process_page(page)

#         # Obtain the LTPage object for the page.
#         layout = device.get_result()

#         extracted_text = ""
#         for element in layout:
#             # print(element)
#             if isinstance(element, LTTextBox) or isinstance(element, LTTextBoxHorizontal):
#                 x, y, text = element.bbox[0], element.bbox[1], element.get_text()
#                 t = 0.05
#                 if x1 - t <= x <= x2 + t and y1 - t <= y <= y2 + t:
#                     extracted_text += text
#         break

#     return extracted_text

# # Test the function with a PDF file and coordinates
# extracted_text = extract_text_by_coordinates(files[0], [97.20097, 533.4017, 540.5599, 565.0175],)
# print(extracted_text)


In [3]:
def levenshtein_distance(str1, str2):
    size_x = len(str1) + 1
    size_y = len(str2) + 1
    matrix = [[0 for j in range(size_y)] for i in range(size_x)]

    for x in range(size_x):
        matrix [x][0] = x
    for y in range(size_y):
        matrix [0][y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if str1[x-1] == str2[y-1]:
                matrix [x][y] = min(
                    matrix[x-1][y] + 1,
                    matrix[x-1][y-1],
                    matrix[x][y-1] + 1
                )
            else:
                matrix [x][y] = min(
                    matrix[x-1][y] + 1,
                    matrix[x-1][y-1] + 1,
                    matrix[x][y-1] + 1
                )
    return matrix[size_x - 1][size_y - 1]

# testing the function
print(levenshtein_distance(etalon, "-  «Капитальный  ремонт  автомобильной  дороги  Р-215  Астрахань-Кочубей- \nКизляр-Махачкала,  подъезд  к  г.  Грозный  на  участке  км  70+127-  км  85+267, \nЧеченская Республика»;\n".lower()))  # should return 3

NameError: ignored

In [ ]:
# from pdfminer.high_level import extract_pages
# from pdfminer.layout import LTTextContainer
# from tqdm import tqdm
# import pandas as pd

# import warnings
# warnings.filterwarnings("ignore")

# # etalon = "Капитальный ремонт автомобильной дороги Р-215 Астрахань - Кочубей - Кизляр - Махачкала, подъезд к г. Грозный на участке км 70+127 – км 85+267, Чеченская Республика".lower()
# import glob
# files = glob.glob('data/general/*.pdf')

# corr_dict = {}
# for file in tqdm(files):
#     print(file)
#     etalon = input('Enter etalon: ').lower()
#     corr_dict[file] = etalon
#     print()
#     # print('\n'*5)

# import pickle
# with open('mapping.pkl', 'wb') as f:
#     pickle.dump(corr_dict, f)

In [4]:
import pickle
with open('mapping.pkl', 'rb') as f:
    corr_dict = pickle.load(f)

In [93]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from tqdm import tqdm
import pandas as pd
from pdfminer.layout import LTTextContainer, LTChar
import re

import warnings
warnings.filterwarnings("ignore")

df = pd.DataFrame(columns=['text', 'label'])
# etalon = "Капитальный ремонт автомобильной дороги Р-215 Астрахань - Кочубей - Кизляр - Махачкала, подъезд к г. Грозный на участке км 70+127 – км 85+267, Чеченская Республика".lower()
import glob
# files = glob.glob('data/general/*.pdf')
# files[0]
files = [
 'data/general/Раздел РИИ №3_ИЭИ-Фрагмент 1.00120-21_КМЭ-25202.pdf',
 'data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-20_СГЭ-04495.pdf',
 'data/general/3864 ИИ 4 К. Изм. 2.02608-19_ГГЭ-07898.pdf',
 'data/general/Раздел ОД №4_Том 4.1.1_Изм.04.02480-21_ГГЭ-04081.pdf',
 'data/general/Раздел ОД №4_Том 4.1.5.02480-21_ГГЭ-04081.pdf',
 'data/general/896-0-ИЭИ-1 изм.2.01499-19_ГГЭ-17661.pdf',
 'data/general/19-15_ОКЭФ_12-ПИР_ИПС-503-19-ИЭИ Изм.2.00484-21_РГЭ-25515.pdf',
 'data/general/01_20_ГК-ПИР_2020-2001-ИЭИ_изм.2.00015-21_РГЭ-25886.pdf',
 'data/general/Кузб-183267_КРАС–ИЭИ1_изм.6.00256-21_КРЭ-26756.pdf',
 'data/general/02_2-15-20 ИЭИ_Изм.5.00158-21_ГГЭ-26767.pdf',
 'data/general/ИЭИ Проектирование оснащения и оснащение ИТСОТБ Верхнего Самарского гидроузла.04955-22_ГГЭ-29629.pdf',
 'data/general/Раздел ИИ 4-ИЭИ1-1-ИЗМ1.07970-22_ГГЭ-36050.pdf',
 'data/general/4735.IV-ИЭИ_изм2.00341-20_ГГЭ-20462.pdf',
 'data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf',
 'data/general/Отчет ИИ №3_Книга №3_(ДПТПиИТ-2020-834-ИЭИ3)_Том 3.3 Фрагмент 1 Изм.1.07430-22_ГГЭ-35753.pdf',
 'data/general/Раздел 1. Раздел II Часть 10_КНИ-ИЭИ изм.2.00127-20_ГГЭ-13675.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 5.7-1.5 Изм.2.00060-20_СПЭ-21903.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 6.2-1.4.1 Изм.1.00072-20_СПЭ-21948.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 6.2-1.4.2 Изм.1.00072-20_СПЭ-21948.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 6.2-1.4.3 Изм.1.00072-20_СПЭ-21948.pdf',
 'data/general/Раздел ПД N1 Экологические изыскания Книга 1.00052-20_СПЭ-18764.pdf',
 'data/general/Раздел ПД N1 Экологические изыскания Книга 2.00052-20_СПЭ-18764.pdf',
 'data/general/Раздел ПД N1 Экологические изыскания.00052-20_СПЭ-18764.pdf',
 'data/general/Раздел ПД №10-Том 10.138К-5-ИЭ-Книга2.00082-20_ГГЭ-06683.pdf',
 'data/general/РС-19-1405-ИЭИ Инженерно-экологические изыскания часть 1 Изм.1 от 14.04.20.00008-20_СКЭ-22197.pdf',
 'data/general/Том3ИЭИ.00321-20_СПЭ-22115.pdf',
 'data/general/ТП 19.066.084-ИЭИ.00011-20_ЕГЭ-21673.pdf',
 'data/general/Раздел ИИ № 4 Том 4.1.07125-22_ГГЭ-35572.pdf',
 'data/general/Раздел ИИ № 4 Том 4.2.07125-22_ГГЭ-35572.pdf',
 'data/general/Раздел ИИ ИЭИ4.1.06920-22_ГГЭ-35136.pdf',
 'data/general/064-2011-Р.pdf']
# files = ['data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf']
corr_files = []
font_id = 0
for file in tqdm(files[::-1]):
    # print(file)
    etalon = corr_dict[file]
    # print(file)
    labels = []
    texts = []
    fonts = []
    squares = []
    ids = []

    for page_layout in extract_pages(file):

        for element in page_layout:
            if isinstance(element, LTTextContainer):
                is_bold = False
                text = element.get_text().replace('\n', '').lower()
                if text.split() != [] and len(text) > 4:
                    texts.append(text)
                    # print(element)
                    end = False
                    for text_line in element:
                        if end: break
                        # print(character)
                        for character in text_line:
                            if isinstance(character, LTChar):
                                # print(character.fontname.lower())
                                if 'bold' in character.fontname.lower():
                                    fonts.append(1)
                                elif 'italic' in character.fontname.lower():
                                    fonts.append(2)
                                else:
                                    fonts.append(0)
                                end = True
                                break

                    if 1 not in labels:
                        if levenshtein_distance(etalon, text) < len(etalon) / 5:
                            labels.append(1)
                        else:
                            labels.append(0)
                    else:
                        labels.append(0)
                    x1, y1, x2, y2 = element.bbox
                    squares.append((int(x2) - int(x1)) * (int(y2) - int(y1)))

                    match = re.search(r'LTTextBoxHorizontal\((\d+)\)', str(element))
                    if match:
                        id = int(match.group(1))  # extract the ID
                        ids.append(id)
        break

    if 1 not in labels:
        corr_files.append(file)
    temp_df = pd.DataFrame({
        'text': texts,
        'label': labels,
        'font': fonts,
        'file': file,
        'squares': squares,
        'ids': ids
    })
    # Append the temporary DataFrame to the master DataFrame
    df = df.append(temp_df, ignore_index=True)
    # print()
    # print('\n'*5)


100%|██████████| 31/31 [00:10<00:00,  2.83it/s]


In [ ]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer, LTChar

for file in corr_files:
    print(file)
    for page_layout in extract_pages(file):
        for element in page_layout:
            print(element)
            # if isinstance(element, LTTextContainer):
            #     for text_line in element:
            #         print(text_line)
                    # for character in text_line:
                    #     if isinstance(character, LTChar):
                    #         print(character.fontname)
                    #         print(character.size)
                    #         break
        break
    print('\n'*5)

In [5]:
import json
from functools import partial
from typing import List
import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
!pip install Levenshtein -qqq
!pip install catboost transformers -qqq
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 73.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.8 MB/s eta 0:00:00


In [ ]:
df[df['file'] == 'data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-20_СГЭ-04495.pdf']
df[df['file'] == 'data/general/4735.IV-ИЭИ_изм2.00341-20_ГГЭ-20462.pdf']

In [136]:
df.to_csv('labeled/up2.csv')

In [177]:
df = pd.read_csv('labeled/up2.csv').drop(columns=['Unnamed: 0'])

In [ ]:
!ls

data  files  labeled  main.ipynb  python-poppler  train.zip


In [ ]:
general = pd.read_csv('labeled/general.csv')
project1 = pd.read_csv('labeled/project1.csv')
project2 = pd.read_csv('labeled/project2.csv')

In [191]:
df[df['font'] == 2].shape

(74, 12)

In [179]:
df[df['font'] == 1].shape

(185, 12)

In [190]:
random_indices = df[df['label'] == 0][df['font'] == 0].sample(n=30).index

# For those indices, set the value in the 'font' column to 1
df.loc[random_indices, 'font'] = 2

In [206]:
# df = pd.concat([general, project1, project2])
# df = pd.concat([general])
df['len_of_text'] = df['text'].apply(len)
# df['len_of_text'] = df['text'].apply(lambda x: len(x.split()))

df['rank'] = df.groupby('file')['len_of_text'].rank(ascending=False, method='min').astype(int)
df['rank_squares'] = df.groupby('file')['squares'].rank(ascending=False, method='min').astype(int)
df['font'] = df['font'].astype(object)  # Convert boolean to int for computation, True will be 1 and False will be 0
df['bold'] = (df['font'] == 1).astype(int)
df['bold_percentage'] = df.groupby('file')['font'].transform(lambda x: x.mean() * 100).astype(int)
# Assuming your DataFrame is named 'df' and the 'id' column is named 'ids'
df['id_percentage'] = df.groupby('file')['ids'].transform(lambda x: (x / x.max()) * 100).astype(int)
df['label'] = df['label'].astype(int)



# 'id_percentage' column now represents the ratio of current id to max id for the same file, multiplied by 100

# 'percentage_true' column now represents the percentage of true bools for each row that has the same file

df.sample(10)

,text,label,font,file,squares,ids,len_of_text,rank,rank_squares,bold,bold_percentage,id_percentage,id_max
328,технический отчет по результатам инженерно-эко...,0,1.0,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,17550.0,1.0,68,2,2,1,100,14,7
144,часть 4. инженерно-экологические изыскания,0,1.0,data/general/Раздел ПД N 1. Пояснительная запи...,6984.0,5.0,43,4,4,1,100,41,12
272,115/21 117/21,0,2.0,data/general/01_20_ГК-ПИР_2020-2001-ИЭИ_изм.2....,962.0,7.0,14,9,9,0,130,77,9
60,дата,0,0.0,data/general/Том3ИЭИ.00321-20_СПЭ-22115.pdf,160.0,8.0,5,9,10,0,60,88,9
263,том 0.4.1,0,1.0,data/general/Кузб-183267_КРАС–ИЭИ1_изм.6.00256...,728.0,6.0,10,7,7,1,87,85,7
198,по результатам инженерно-экологических изыскан...,0,2.0,data/general/Отчет ИИ №3_Книга №3_(ДПТПиИТ-202...,11016.0,4.0,87,4,3,0,88,50,8
32,проектно-изыскательский институт электрификаци...,0,2.0,data/general/Раздел ИИ № 4 Том 4.1.07125-22_ГГ...,13545.0,0.0,138,1,2,0,122,0,8
280,из № 1 503/1 503/2 2 3 503/3,0,2.0,data/general/19-15_ОКЭФ_12-ПИР_ИПС-503-19-ИЭИ ...,3245.0,5.0,33,3,5,0,111,62,8
66,проектная документация,0,0.0,data/general/РС-19-1405-ИЭИ Инженерно-экологич...,2884.0,4.0,22,7,5,0,83,30,13
292,москва 2018,0,1.0,data/general/896-0-ИЭИ-1 изм.2.01499-19_ГГЭ-17...,936.0,8.0,12,7,7,1,88,100,8


In [248]:
train_df = df.drop(columns=[ 'text', 'squares', 'id_max', 'ids', 'len_of_text', 'bold'])
train_df.sample(5)

,label,font,file,rank,rank_squares,bold_percentage,id_percentage
134,0,2.0,data/general/Раздел ПД N 1. Пояснительная запи...,12,12,115,66
43,0,0.0,data/general/ТП 19.066.084-ИЭИ.00011-20_ЕГЭ-21...,2,2,63,20
310,0,2.0,data/general/Раздел ОД №4_Том 4.1.1_Изм.04.024...,12,14,100,63
81,0,1.0,data/general/Раздел ПД №10-Том 10.138К-5-ИЭ-Кн...,8,7,84,58
147,0,0.0,data/general/Раздел ПД N 1. Пояснительная запи...,12,12,100,66


In [249]:
from sklearn.model_selection import StratifiedKFold, GroupKFold

# kfold = GroupKFold(n_splits=5, shuffle=True, random_state=0xFACED)
kfold = GroupKFold(n_splits=5)

metrics = []
models = []
fold_to_train = 0
for fold, (train_index, val_index) in enumerate(kfold.split(X=train_df,groups= train_df['file'])):
    # if fold == fold_to_train:
        X_train = train_df.iloc[train_index].drop(columns=['file'])
        X_val = train_df.iloc[val_index].drop(columns=['file'])
        print(X_train.columns, X_val.shape)

        y_train = X_train["label"]
        y_val = X_val["label"]
        X_train = X_train.drop(["label"], axis=1)
        X_val = X_val.drop(["label"], axis=1)

        train_pool = Pool(
            data=X_train,
            label=y_train,
            # text_features=text_features
        )
        eval_pool = Pool(
            data=X_val,
            label=y_val,
            # text_features=text_features
        )

        model = CatBoostClassifier(eval_metric='F1')

        model.fit(
            train_pool,
            eval_set=eval_pool,
            verbose=True,
            metric_period=100,
            use_best_model=True,
            early_stopping_rounds=200
        )

        models.append(model)

        # X_val["scores"] = model.predict_proba(X_val[feats])[:, 1]

        # y_val = features.iloc[val_index][["target", "variantid1", "variantid2"]]
        # pr_auc_macro_metr = pr_auc_macro(
        #     target_df=y_val,
        #     predictions_df=X_val,
        #     prec_level=0.75,
        #     cat_column="cat3_grouped"
        # )
        # print(f'Fold #{fold}: {pr_auc_macro_metr}')
        # metrics.append(pr_auc_macro_metr)

Index(['label', 'font', 'rank', 'rank_squares', 'bold_percentage',
       'id_percentage'],
      dtype='object') (68, 6)
Learning rate set to 0.023016
0:	learn: 0.9545455	test: 0.9090909	best: 0.9090909 (0)	total: 1.16ms	remaining: 1.16s
100:	learn: 0.9545455	test: 0.9090909	best: 1.0000000 (2)	total: 70.7ms	remaining: 629ms


200:	learn: 1.0000000	test: 0.9090909	best: 1.0000000 (2)	total: 150ms	remaining: 596ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 1
bestIteration = 2

Shrink model to first 3 iterations.
Index(['label', 'font', 'rank', 'rank_squares', 'bold_percentage',
       'id_percentage'],
      dtype='object') (73, 6)
Learning rate set to 0.022912
0:	learn: 0.9302326	test: 0.9230769	best: 0.9230769 (0)	total: 846us	remaining: 845ms
100:	learn: 0.9545455	test: 0.9230769	best: 1.0000000 (3)	total: 77.1ms	remaining: 686ms


200:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (3)	total: 157ms	remaining: 626ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 1
bestIteration = 3

Shrink model to first 4 iterations.
Index(['label', 'font', 'rank', 'rank_squares', 'bold_percentage',
       'id_percentage'],
      dtype='object') (67, 6)
Learning rate set to 0.023037
0:	learn: 0.9565217	test: 0.8888889	best: 0.8888889 (0)	total: 1.07ms	remaining: 1.07s
100:	learn: 0.9787234	test: 0.8000000	best: 0.8888889 (0)	total: 66.7ms	remaining: 593ms


200:	learn: 1.0000000	test: 0.8000000	best: 0.8888889 (0)	total: 146ms	remaining: 579ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8888888889
bestIteration = 0

Shrink model to first 1 iterations.
Index(['label', 'font', 'rank', 'rank_squares', 'bold_percentage',
       'id_percentage'],
      dtype='object') (67, 6)
Learning rate set to 0.023037
0:	learn: 0.9545455	test: 0.9090909	best: 0.9090909 (0)	total: 1.03ms	remaining: 1.03s
100:	learn: 1.0000000	test: 0.9090909	best: 0.9090909 (0)	total: 64.6ms	remaining: 575ms


200:	learn: 1.0000000	test: 0.9090909	best: 0.9090909 (0)	total: 133ms	remaining: 527ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.9090909091
bestIteration = 0

Shrink model to first 1 iterations.
Index(['label', 'font', 'rank', 'rank_squares', 'bold_percentage',
       'id_percentage'],
      dtype='object') (67, 6)
Learning rate set to 0.023037
0:	learn: 0.9302326	test: 1.0000000	best: 1.0000000 (0)	total: 948us	remaining: 947ms
100:	learn: 0.9545455	test: 1.0000000	best: 1.0000000 (0)	total: 70.9ms	remaining: 631ms


200:	learn: 1.0000000	test: 1.0000000	best: 1.0000000 (0)	total: 144ms	remaining: 572ms
Stopped by overfitting detector  (200 iterations wait)

bestTest = 1
bestIteration = 0

Shrink model to first 1 iterations.


In [ ]:
train_df.iloc[val_index].file.unique()

array(['data/general/ТП 19.066.084-ИЭИ.00011-20_ЕГЭ-21673.pdf',
       'data/general/РС-19-1405-ИЭИ Инженерно-экологические изыскания часть 1 Изм.1 от 14.04.20.00008-20_СКЭ-22197.pdf',
       'data/general/Раздел ПД N1 Экологические изыскания Книга 2.00052-20_СПЭ-18764.pdf',
       'data/general/Раздел ПД N 1. Пояснительная записка Том 6.2-1.4.3 Изм.1.00072-20_СПЭ-21948.pdf',
       'data/general/4735.IV-ИЭИ_изм2.00341-20_ГГЭ-20462.pdf',
       'data/general/19-15_ОКЭФ_12-ПИР_ИПС-503-19-ИЭИ Изм.2.00484-21_РГЭ-25515.pdf',
       'data/general/Раздел РИИ №3_ИЭИ-Фрагмент 1.00120-21_КМЭ-25202.pdf'],
      dtype=object)

In [ ]:
# data/general/Раздел ИИ ИЭИ4.1.06920-22_ГГЭ-35136.pdf
# data/general/Раздел 1. Раздел II Часть 10_КНИ-ИЭИ изм.2.00127-20_ГГЭ-13675.pdf
# data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf
# data/general/Раздел ИИ 4-ИЭИ1-1-ИЗМ1.07970-22_ГГЭ-36050.pdf


In [102]:
df[df['file'] == path]

,text,label,font,file,squares,ids,len_of_text,rank,rank_squares,bold,bold_percentage,id_percentage


In [196]:
files = ['data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf']
# files = df.file.unique()
for path in tqdm(files):
    temp_df = df[df['file'] == path].reset_index(drop=True)
    test_pool = Pool(
        data=temp_df.drop(columns=['label', 'file', 'text', 'squares', 'ids', 'len_of_text', 'bold']),
        # text_features=text_features
    )
    preds = []
    for model in models:
        preds.append(model.predict_proba(test_pool)[:, 1])
    temp_df['pred']  = np.mean(preds, axis=0)
    if (temp_df.iloc[temp_df['pred'].idxmax(), 1]) == 0:
        print(path)
        # print(temp_df.iloc[temp_df['pred'].idxmax()])
        # break

100%|██████████| 1/1 [00:00<00:00, 129.65it/s]

data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf


In [197]:
# italic, отчет, stupid, bad pdf reading

In [198]:
temp_df

,text,label,font,file,squares,ids,len_of_text,rank,rank_squares,bold,bold_percentage,id_percentage,pred
0,общество с ограниченной ответственностью научн...,0,2.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,12512.0,0.0,91,2,3,0,33,0,0.422350
1,(ооо нпп «изыскатель»),0,0.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,2296.0,1.0,23,7,7,0,33,9,0.424133
2,регистрационный номер в реестре членов сро №13...,0,2.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,5754.0,2.0,64,5,5,0,33,18,0.429294
3,заказчик – ооо «пермтотинефть»,0,0.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,3122.0,3.0,31,6,6,0,33,27,0.420076
4,«строительство и обустройство скважин куста № ...,1,1.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,13410.0,4.0,79,3,2,1,33,36,0.429754
5,технический отчет по результатам инженерно-гео...,0,2.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,20069.0,5.0,94,1,1,0,33,45,0.481646
6,приложение н.2 к разделу 1 «пояснительная запи...,0,0.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,10488.0,6.0,70,4,4,0,33,54,0.419379
7,2019/021-игди2,0,0.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,1484.0,7.0,15,9,8,0,33,63,0.405708
8,том 1.2,0,2.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,686.0,8.0,8,11,11,0,33,72,0.419875
9,изм. № док. подп.,0,0.0,data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf,1332.0,9.0,20,8,10,0,33,81,0.405732


In [92]:
files = [
 'data/general/Раздел РИИ №3_ИЭИ-Фрагмент 1.00120-21_КМЭ-25202.pdf',
 'data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-20_СГЭ-04495.pdf',
 'data/general/3864 ИИ 4 К. Изм. 2.02608-19_ГГЭ-07898.pdf',
 'data/general/Раздел ОД №4_Том 4.1.1_Изм.04.02480-21_ГГЭ-04081.pdf',
 'data/general/Раздел ОД №4_Том 4.1.5.02480-21_ГГЭ-04081.pdf',
 'data/general/896-0-ИЭИ-1 изм.2.01499-19_ГГЭ-17661.pdf',
 'data/general/19-15_ОКЭФ_12-ПИР_ИПС-503-19-ИЭИ Изм.2.00484-21_РГЭ-25515.pdf',
 'data/general/01_20_ГК-ПИР_2020-2001-ИЭИ_изм.2.00015-21_РГЭ-25886.pdf',
 'data/general/Кузб-183267_КРАС–ИЭИ1_изм.6.00256-21_КРЭ-26756.pdf',
 'data/general/02_2-15-20 ИЭИ_Изм.5.00158-21_ГГЭ-26767.pdf',
 'data/general/ИЭИ Проектирование оснащения и оснащение ИТСОТБ Верхнего Самарского гидроузла.04955-22_ГГЭ-29629.pdf',
 'data/general/Раздел ИИ 4-ИЭИ1-1-ИЗМ1.07970-22_ГГЭ-36050.pdf',
 'data/general/4735.IV-ИЭИ_изм2.00341-20_ГГЭ-20462.pdf',
 'data/general/╨в╨╛╨╝ 1.2.00001-21_╨Х╨У╨н-26404.pdf',
 'data/general/Отчет ИИ №3_Книга №3_(ДПТПиИТ-2020-834-ИЭИ3)_Том 3.3 Фрагмент 1 Изм.1.07430-22_ГГЭ-35753.pdf',
 'data/general/Раздел 1. Раздел II Часть 10_КНИ-ИЭИ изм.2.00127-20_ГГЭ-13675.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 5.7-1.5 Изм.2.00060-20_СПЭ-21903.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 6.2-1.4.1 Изм.1.00072-20_СПЭ-21948.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 6.2-1.4.2 Изм.1.00072-20_СПЭ-21948.pdf',
 'data/general/Раздел ПД N 1. Пояснительная записка Том 6.2-1.4.3 Изм.1.00072-20_СПЭ-21948.pdf',
 'data/general/Раздел ПД N1 Экологические изыскания Книга 1.00052-20_СПЭ-18764.pdf',
 'data/general/Раздел ПД N1 Экологические изыскания Книга 2.00052-20_СПЭ-18764.pdf',
 'data/general/Раздел ПД N1 Экологические изыскания.00052-20_СПЭ-18764.pdf',
 'data/general/Раздел ПД №10-Том 10.138К-5-ИЭ-Книга2.00082-20_ГГЭ-06683.pdf',
 'data/general/РС-19-1405-ИЭИ Инженерно-экологические изыскания часть 1 Изм.1 от 14.04.20.00008-20_СКЭ-22197.pdf',
 'data/general/Том3ИЭИ.00321-20_СПЭ-22115.pdf',
 'data/general/ТП 19.066.084-ИЭИ.00011-20_ЕГЭ-21673.pdf',
 'data/general/Раздел ИИ № 4 Том 4.1.07125-22_ГГЭ-35572.pdf',
 'data/general/Раздел ИИ № 4 Том 4.2.07125-22_ГГЭ-35572.pdf',
 'data/general/Раздел ИИ ИЭИ4.1.06920-22_ГГЭ-35136.pdf',
 'data/general/064-2011-Р.pdf']

In [93]:
import json
from functools import partial
from typing import List
import glob
from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
!pip install Levenshtein -qqq
!pip install catboost transformers -qqq
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances
from sklearn.model_selection import train_test_split

In [95]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextContainer
from tqdm import tqdm
import pandas as pd
from pdfminer.layout import LTTextContainer, LTChar
import re
import glob
import warnings
warnings.filterwarnings("ignore")


def extract_test_features(file):
    print(file)
    texts = []
    fonts = []
    squares = []
    ids = []
    coords = []
    relative_coords = []
    for page_layout in extract_pages(file):
        _x1, _y1, _x2, _y2 = page_layout.bbox
        for i, element in enumerate(page_layout):
            if isinstance(element, LTTextContainer):
                text = element.get_text().replace('\n', '')

                if '(cid:' in text:
                    return [], 'Files encoding is corrupted'

                if text.split() != [] and len(text) > 4:
                    texts.append(text)

                    end = False
                    for text_line in element:
                        if end: break
                        for character in text_line:
                            if isinstance(character, LTChar):
                                if 'bold' in character.fontname.lower():
                                    fonts.append(1)
                                elif 'italic' in character.fontname.lower():
                                    fonts.append(2)
                                else:
                                    fonts.append(0)
                                end = True
                                break

                    x1, y1, x2, y2 = element.bbox
                    coords.append([x1, y1, x2, y2])
                    relative_coords.append([x1 / _x2, y1 / _y2, (x2 - x1) / _x2, (y2 - y1) / _y2])

                    squares.append((int(x2) - int(x1)) * (int(y2) - int(y1)))

                    match = re.search(r'LTTextBoxHorizontal\((\d+)\)', str(element))
                    if match:
                        id = int(match.group(1))
                        ids.append(id)
        break

    if not texts: return [], 'File consists of scans'
    if len(texts) < 3: return [], 'Main page consists of scan'
    if len(texts) > 25: return [], 'Seems like incorrect reading'

    test_df = pd.DataFrame({
        'text': texts,
        'font': fonts,
        'file': file,
        'squares': squares,
        'ids': ids,
        'coords': coords,
        'relative_coords': relative_coords
    })
    return test_df, True


def create_test_features(df):
    df['len_of_text'] = df['text'].apply(len)
    # df['len_of_text'] = df['text'].apply(lambda x: len(x.split()))

    df['rank'] = df.groupby('file')['len_of_text'].rank(ascending=False, method='min').astype(int)
    df['rank_squares'] = df.groupby('file')['squares'].rank(ascending=False, method='min').astype(int)
    df['font'] = df['font'].astype(object)  # Convert boolean to int for computation, True will be 1 and False will be 0
    df['bold'] = (df['font'] == 1).astype(int)
    df['bold_percentage'] = df.groupby('file')['font'].transform(lambda x: x.mean() * 100).astype(int)
    df['id_percentage'] = df.groupby('file')['ids'].transform(lambda x: (x / x.max()) * 100).astype(int)

    return df

# test_df, result = extract_test_features(corrupted_files[5])
# if test_df:
#     test_df = create_test_features(test_df)
# else:
#     print(result)

In [96]:
def extract_test_features(file):
    print(file)
    texts = []
    fonts = []
    squares = []
    ids = []
    coords = []
    relative_coords = []
    for page_layout in extract_pages(file):
        _x1, _y1, _x2, _y2 = page_layout.bbox
        for i, element in enumerate(page_layout):
            if isinstance(element, LTTextContainer):
                text = element.get_text().replace('\n', '')

                if '(cid:' in text:
                    return [], 'Files encoding is corrupted'

                if text.split() != [] and len(text) > 4:
                    texts.append(text)

                    end = False
                    for text_line in element:
                        if end: break
                        for character in text_line:
                            if isinstance(character, LTChar):
                                if 'bold' in character.fontname.lower():
                                    fonts.append(1)
                                elif 'italic' in character.fontname.lower():
                                    fonts.append(2)
                                else:
                                    fonts.append(0)
                                end = True
                                break

                    x1, y1, x2, y2 = element.bbox
                    coords.append([x1, y1, x2, y2])
                    relative_coords.append([x1 / _x2, y1 / _y2, (x2 - x1) / _x2, (y2 - y1) / _y2])

                    squares.append((int(x2) - int(x1)) * (int(y2) - int(y1)))

                    match = re.search(r'LTTextBoxHorizontal\((\d+)\)', str(element))
                    if match:
                        id = int(match.group(1))
                        ids.append(id)
        break

    if not texts: return [], 'File consists of scans'
    if len(texts) < 3: return [], 'Main page consists of scan'
    if len(texts) > 25: return [], 'Seems like incorrect reading'

    test_df = pd.DataFrame({
        'text': texts,
        'font': fonts,
        'file': file,
        'squares': squares,
        'ids': ids,
        'coords': coords,
        'relative_coords': relative_coords
    })
    return test_df, True

In [10]:
for page_layout in extract_pages(files[1]):
    # for element in page_layout:
    break

(0, 0, 595, 842)


In [7]:
columns_to_use = ['font', 'rank', 'rank_squares', 'bold_percentage', 'id_percentage']
checkpoint_name = 'checkpoints/models.pkl'

In [8]:
import pickle

def inference_models(checkpoint_name, test_df):
    with open(checkpoint_name, 'rb') as f:
        models = pickle.load(f)

    test_pool = Pool(
        data=test_df[columns_to_use]
    )
    preds = []
    for model in models:
        preds.append(model.predict_proba(test_pool)[:, 1])
    test_df['pred']  = np.mean(preds, axis=0)
    return test_df, test_df.loc[test_df['pred'].idxmax(), 'text'].strip()

In [9]:
import time

In [79]:
import Levenshtein as lev

def calculate_distances(target, list_of_strings):
    target_length = len(target.split())
    distances = {}

    for string in list_of_strings:
        string_words = string.split()

        # If the string has at least as many words as the target
        if len(string_words) >= target_length:
            for i in range(len(string_words) - target_length + 1):
                window = " ".join(string_words[i:i+target_length])
                distance = lev.distance(target, window)

                # Save the distance for this window
                distances[window] = (distance / len(target)) * 100
        else:
            # If the string has fewer words than the target
            distance = lev.distance(target, string)
            distances[string] = (distance / len(target)) * 100

    return distances

In [ ]:
import Levenshtein as lev
import math

def calculate_distances(target, list_of_strings, stride_fraction=1/4, threshold=0.3):
    target_length = len(target.split())
    min_distances = []

    stride_length = math.ceil(target_length * stride_fraction)

    for string in list_of_strings:
        all_distances = []
        string_words = string.split()

        if len(string_words) > target_length:
            i = 0
            while i < len(string_words) - target_length + 1:
                window = " ".join(string_words[i:i+target_length])

                distance = lev.distance(target, window) / len(target)
                if distance < threshold:
                    for j in range(max(i-target_length, 0), min(i + target_length, len(string_words) - target_length + 1)):
                        detailed_window = " ".join(string_words[j:j+target_length])
                        detailed_distance = lev.distance(target, detailed_window) / len(target)

                        all_distances.append((detailed_window, detailed_distance * 100))
                    i += stride_length
                else:
                    i += stride_length
        else:
            dist = lev.distance(target, string) / len(target)
            all_distances.append((string, dist * 100))

        if all_distances:
            min_window = min(all_distances, key=lambda x: x[1])
            min_distances.append([min_window[0],  min_window[1]])

    return min_distances


In [97]:
import Levenshtein as lev
import math

def calculate_distances(target, list_of_strings, stride_fraction=1/4, threshold=0.3):
    target_length = len(target.split())
    min_distances = []

    # Calculate stride length based on fraction of target length
    stride_length = math.ceil(target_length * stride_fraction)

    for string in list_of_strings:
        all_distances = []
        string_words = string.split()

        # If the string has at least as many words as the target
        # print(len(string_words), target_length)
        if len(string_words) > target_length:
            i = 0
            while i < len(string_words) - target_length + 1:
                window = " ".join(string_words[i:i+target_length])

                distance = lev.distance(target, window) / len(target)
                # print(window, distance)
                if distance < threshold:
                    for j in range(max(i-target_length, 0), min(i + target_length, len(string_words) - target_length + 1)):
                        detailed_window = " ".join(string_words[j:j+target_length])
                        detailed_distance = lev.distance(target, detailed_window) / len(target)

                        all_distances.append((detailed_window, detailed_distance * 100))
                        # print(all_distances)
                    i += stride_length  # move with larger stride
                else:
                    i += stride_length  # move with larger stride
        else:
            dist = lev.distance(target, string) / len(target)
            # print(dist)
            all_distances.append((string, dist * 100))

    # # Find minimum distances among every three consecutive windows
    #     for i in range(0, maxlen(all_distances)-target_length+1, target_length):
    #         windows = all_distances[i:i+target_length]
        # print(all_distances)
        if all_distances:  # ensure windows is not empty
            min_window = min(all_distances, key=lambda x: x[1])
            min_distances.append([min_window[0],  min_window[1]])  # update the dictionary with the minimum distance window

    return min_distances


In [99]:
!pip install spacy fuzzywuzzy python-Levenshtein


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [126]:
import spacy
import re
from fuzzywuzzy import fuzz
import Levenshtein

nlp = spacy.load('ru_core_news_sm')

import re

def remove_special_characters(string):
    return re.sub(r'\W', '', string)


def difference_type(word1, word2):
    if word1 == word2:
        return None  # слова совпадают, пропускаем их

    if remove_special_characters(word1) == remove_special_characters(word2):
        return "Missing special character"

    if word1.lower() == word2.lower():
        return "Just different capital"

    if word1.isdigit() and word2.isdigit():
        if abs(int(word1) - int(word2)) < 10:
            return 'Numerical difference'
        else:
            return 'Large numerical difference'
    token1 = nlp(word1)[0]
    token2 = nlp(word2)[0]
    if token1.lemma_ == token2.lemma_:
        if token1.pos_ != token2.pos_:
            return 'Different word forms'
        else:
            return 'Same lemma, different inflection'

    if Levenshtein.distance(word1, word2) <= 2:
        return 'Possible spelling mistake'

    # Using Spacy for lemmatization and part-of-speech tagging


    return 'Different words'

def compare_strings(str1, str2):
    words1 = str1.split()
    words2 = str2.split()

    # Find words that are only in one of the strings
    words1_only = set(words1) - set(words2)
    words2_only = set(words2) - set(words1)

    differences = []
    mn_len = min(len(words1), len(words2))
    for i in range(mn_len):
        difference = difference_type(words1[i], words2[i])
        differences.append((words1[i], words2[i], difference))

    for word in words1_only:
        differences.append((word, None, 'Word only in first string'))

    for word in words2_only:
        differences.append((None, word, 'Word only in second string'))

    return differences

str1 = "я играю 123"
str2 = "ты играешь 124 и что-то еще"
print(compare_strings(str1, str2))


[('я', 'ты', 'Possible spelling mistake'), ('играю', 'играешь', 'Same lemma, different inflection'), ('123', '124', 'Numerical difference'), ('123', None, 'Word only in first string'), ('я', None, 'Word only in first string'), ('играю', None, 'Word only in first string'), (None, 'ты', 'Word only in second string'), (None, 'и', 'Word only in second string'), (None, 'еще', 'Word only in second string'), (None, 'играешь', 'Word only in second string'), (None, '124', 'Word only in second string'), (None, 'что-то', 'Word only in second string')]


In [127]:
s1 = '«Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»'
s2 = '«Реконструкции платины Горельского гидроузла на р Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»'
compare_strings(s1, s2)

[('«Реконструкция', '«Реконструкции', 'Same lemma, different inflection'),
 ('плотины', 'платины', 'Possible spelling mistake'),
 ('Горельского', 'Горельского', None),
 ('гидроузла', 'гидроузла', None),
 ('на', 'на', None),
 ('р.', 'р', 'Missing special character'),
 ('Цна,', 'Цна,', None),
 ('с.', 'с.', None),
 ('Горелое,', 'Горелое,', None),
 ('Тамбовский', 'Тамбовский', None),
 ('район,', 'район,', None),
 ('Тамбовская', 'Тамбовская', None),
 ('область.', 'область.', None),
 ('(Корректировка', '(Корректировка', None),
 ('в', 'в', None),
 ('том', 'том', None),
 ('числе', 'числе', None),
 ('проектные', 'проектные', None),
 ('и', 'и', None),
 ('изыскательские', 'изыскательские', None),
 ('работы)»', 'работы)»', None),
 ('«Реконструкция', None, 'Word only in first string'),
 ('р.', None, 'Word only in first string'),
 ('плотины', None, 'Word only in first string'),
 (None, 'платины', 'Word only in second string'),
 (None, '«Реконструкции', 'Word only in second string'),
 (None, 'р', 'Wo

In [ ]:
!python -m spacy download ru_core_news_sm


In [172]:
s1 = 'Комплекс  работ  в  ходе  инженерно-экологических  изысканий  для  объекта «Реконструкция  плотины  Горельского  гидроузла  на  р.  Цна,  с.  Горелое,  Тамбовский  район, Тамбовская  область.  (Корректировка  в  том  числе  проектные  и  изыскательские  работы)»  для ГФУ  «Цнинская  шлюзованная  система»  выполнен  в  июне  2019  г.  Основанием  для производства  работ  является  договор  №  19/119-И,  техническое  задание  на  выполнение инженерно-экологических изысканий. '

In [171]:
s2 = '«Реконструкция  плотины Горельского  гидроузла  на  р.  Цна,  с.  Горелое,  Тамбовский  район,  Тамбовская  область. (Корректировка  в  том  числе  проектные  и  изыскательские  работы)»  разработаны  в соответствии  с  требованиями  СП  47.13330.2016  «Инженерные  изыскания  для  строительства. Основные  положения».  Актуализированная  редакция  [16];  СП  11-102-97  «Инженерно-экологические  изыскания  для  строительства»  [17],  Постановлением  Правительства  РФ  от 26.12.2014  г.  №  1521  [7]  и  на  основании  других  действующих  законодательных  актов  и нормативных документов. '

In [186]:
distances = calculate_distances(target, [s1, s2])
distances

[['«Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»',
  9.392265193370166],
 ['«Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»',
  9.392265193370166]]

In [183]:
for window, distance in distances:
    if distance / len(target) < 0.2:
        print(i)
        print(window)
        print(distance)


8
«Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»
9.392265193370166
8
«Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»
9.392265193370166


In [44]:
target

'«Строительство скоростной автомобильной дороги  Москва – Санкт-Петербург на участке км 58 – км 684  (с последующей эксплуатацией на платной основе),  3 этап км 149 – км 208»'

In [155]:
target

'Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные Корректировкаи изыскательские работы)'

In [11]:
import re

def replace_multiple_spaces(text):
    return re.sub(' +', ' ', text)

target = replace_multiple_spaces(target)

NameError: ignored

In [87]:
for file in files[2:3]:
    changed = []
    s = time.time()
    test_df, result = extract_test_features(file)

    if isinstance(test_df, pd.DataFrame):
        test_df = create_test_features(test_df)
    else:
        print(result)

    result_df, target = inference_models(checkpoint_name, test_df)
    target = replace_multiple_spaces(target)
    # target = '«Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»'
    from pdfminer.high_level import extract_pages
    result = []
    for i, page_layout in enumerate(tqdm(extract_pages(file))):
        # if i != 1: continue
        _x1, _y1, _x2, _y2 = page_layout.bbox
        texts = []
        relative_coords = []
        for element in page_layout:
            if isinstance(element, LTTextContainer):
                # print(element.get_text())
                x1, y1, x2, y2 = element.bbox
                text = replace_multiple_spaces(element.get_text().replace('\n', ' ').strip())
                if len(text) > 3:
                    relative_coords.append(([x1 / _x2, y1 / _y2, (x2 - x1) / _x2, (y2 - y1) / _y2]))
                    texts.append(text)

        distances = calculate_distances(target, texts)

        for window, distance in distances.items():
            if distance / len(target) < 0.2:

                for j in range(len(texts)):
                    if window in texts[j]:
                        # changed.append((texts[j], texts[j].replace(window, target)))
                        rel_coord = relative_coords[j]
                        break

                result.append((i, window, rel_coord, distance / len(target) ))
        # break
    print(file)
    print('It took ', int(time.time() - s), 's')
    print()

data/general/3864 ИИ 4 К. Изм. 2.02608-19_ГГЭ-07898.pdf


134it [00:15,  8.61it/s]

data/general/3864 ИИ 4 К. Изм. 2.02608-19_ГГЭ-07898.pdf
It took  16 s



In [89]:
changed[0][0]

IndexError: ignored

In [90]:
changed[0][1]

IndexError: ignored

In [91]:
target

'«Строительство и реконструкция автомобильной дороги М-5 «Урал» - от Москвы через Рязань, Пензу, Самару, Уфу до Челябинска Строительство транспортной развязки на км 974 автомобильной дороги М-5 «Урал» - от Москвы через Рязань, Пензу, Самару, Уфу до Челябинска, Самарская область. Корректировка».'

In [88]:
result

[(0,
  '«Строительство и реконструкция автомобильной дороги М-5 «Урал» - от Москвы через Рязань, Пензу, Самару, Уфу до Челябинска Строительство транспортной развязки на км 974 автомобильной дороги М-5 «Урал» - от Москвы через Рязань, Пензу, Самару, Уфу до Челябинска, Самарская область. Корректировка».',
  [0.1580729691594436,
   0.5093864500190042,
   0.764091177853927,
   0.12813568985176751],
  0.0),
 (1,
  '«Строительство и реконструкция автомобильной дороги М-5 «Урал» - от Москвы через Рязань, Пензу, Самару, Уфу до Челябинска Строительство транспортной развязки на км 974 автомобильной дороги М-5 «Урал» - от Москвы через Рязань, Пензу, Самару, Уфу до Челябинска, Самарская область. Корректировка».',
  [0.1580729691594436,
   0.5273454009882174,
   0.7639272996035744,
   0.12827822120866605],
  0.0),
 (131,
  '«Строительство и реконструкция автомобильной дороги М-5 «Урал» -от Москвы через Рязань, Пензу, Самару, Уфу до Челябинска Строительство транспортной развязки на км 974 автомобиль

In [84]:
len(result)

1

In [191]:
target = '«Реконструкция плотины Горельского гидроузла на р. Цна, с. Горелое, Тамбовский район, Тамбовская область. (Корректировка в том числе проектные и изыскательские работы)»'

In [59]:
result_df

,text,font,file,squares,ids,coords,relative_coords,pages,len_of_text,rank,rank_squares,bold,bold_percentage,id_percentage,pred
0,Реконструкция плотины Горельского гидроузла н...,1,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,56172,0,"[95.2800342756015, 532.9594999589843, 548.0043...","[0.16013451138756554, 0.6329685272672023, 0.76...",0,182,1,1,1,71,0,0.529246
1,Технический отчет по результатам инженерно-эко...,1,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,17550,1,"[98.3995963384576, 446.1314000011057, 548.9640...","[0.16537747283774387, 0.5298472684098643, 0.75...",1,68,2,2,1,71,14,0.457483
2,19/119-И-ИЭИ,1,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,2250,2,"[260.76, 367.8500000007152, 385.98260631509044...","[0.4382521008403361, 0.43687648456141953, 0.21...",2,14,3,3,1,71,28,0.457483
3,Том 3,1,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,918,3,"[295.438, 283.1300000007152, 346.7355896315769...","[0.49653445378151256, 0.33625890736426983, 0.0...",3,6,5,4,1,71,42,0.463904
4,35-20,0,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,286,5,"[414.9242006103037, 128.87693584416243, 440.58...","[0.697351597664376, 0.15306049387667747, 0.043...",5,5,6,7,0,71,71,0.447048
5,2921.04.2020,0,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,600,6,"[507.7139916413332, 125.8110002762967, 557.213...","[0.853300826287955, 0.14941924023313147, 0.083...",6,12,4,6,0,71,85,0.460314
6,2019,1,data/general/Отчет ИЭИ 19-119-И-3 изм1.00020-2...,738,7,"[318.6, 24.410600000715224, 359.0983811252344,...","[0.5354621848739496, 0.028991211402274612, 0.0...",7,5,6,5,1,71,100,0.447048
